In [2]:
from random import randrange
from GameAPI import GameAPI
from os import path

import numpy as np
import pprint
import time

teamId = 1257
worldId = 0
startWorld = 4
endWorld = 10
dimension = 40
alpha = 0.9

GameAPI = GameAPI()

def load_gridWorld(worldId):
    gridWorld = np.load('world' + str(worldId) + '.npy')
    return gridWorld

def load_qVals(worldId):
    qVals = np.load('world' + str(worldId) + 'qVals.npy', allow_pickle=True)
    return qVals

def before_exit(worldId, gridWorld, qValues):
    np.save('world' + str(worldId), gridWorld)
    np.save('world' + str(worldId) + 'qVals', qValues)

def init_q_values(dimension):
    qValues = []
    for i in range(dimension):
        for j in range(dimension):
            tempDict = {'N':0, 'S':0, 'E':0, 'W':0, 'i':i, 'j':j}
            qValues.append(tempDict)
    return qValues

def get_action(x, y, qValue):
    actions = ['N', 'S', 'E', 'W']
    if(x == 0):
        actions.remove('W')
    if (x == dimension - 1):
        actions.remove('E')
    if (y == 0):
        actions.remove('S')
    if (y == dimension - 1):
        actions.remove('N')

    # if(qValue['N'] == 0):
    #     return 'N'
    # if (qValue['S'] == 0):
    #     return 'S'
    # if (qValue['E'] == 0):
    #     return 'E'
    # if (qValue['W'] == 0):
    #     return 'W'

    return actions[randrange(0, len(actions), 1)]

def q_learning(gridWorld, qValues):
    curX = 0
    curY = 0

    # Update location and worldId
    location = GameAPI.locate_me(teamId)
    worldId = int(location['world'])
    state = location['state'].split(':')
    curX = int(state[0])
    curY = int(state[1])

    while True:
        # for act in actions:
        act = get_action(curX, curY, qValues[(curX+1)*(curY+1)-1])
        # print(curX, curY, act)
        move_res = GameAPI.make_move(teamId, worldId, act)
        reward = move_res['reward']
        scoreIncrement = move_res['scoreIncrement']
        newState = move_res['newState']

        # If reached the end, save the values and return
        if(newState == 'null' or newState == None):
            before_exit(worldId, gridWorld, qValues)
            print('terminal state:', curX, curY, act)
            return
        else:
            newX = int(newState['x'])
            newY = int(newState['y'])
            qValues[(curX+1)*(curY+1)-1][act] = (1 - alpha) * qValues[(curX+1)*(curY+1)-1][act] + alpha * (reward + scoreIncrement)

        # Update location
        curX = newX
        curY = newY

qValues = []
gridWorld = []

for i in range(startWorld, endWorld+1):
    worldId = i

    for j in range(0, 5):
        print('world: ', worldId, 'time: ', j)
        GameAPI.enter_world(teamId, worldId)
        # If files exist, load them, if not initialize the world
        if(path.exists('world' + str(worldId) + '.npy')):
            qValues = load_qVals(worldId)
            gridWorld = load_gridWorld(worldId)
        else:
            qValues = init_q_values(dimension)
            np.zeros((dimension,dimension), dtype=float)

        q_learning(gridWorld, qValues)

    before_exit(worldId, gridWorld, qValues)

ModuleNotFoundError: No module named 'GameAPI'